### Zigbang 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

In [5]:
addr = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기

In [6]:
import geohash2

In [10]:
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

#### 3. geohash로 매물 아이디 가져오기

In [19]:
url = f'https://apis.zigbang.com/v2/items/oneroom?geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&domain=zigbang&checkAnyItemWithoutFilter=true'
response = requests.get(url)
item_ids = [data['itemId'] for data in response.json()['items']]
len(item_ids), item_ids[:5]

(381, [42196796, 42078100, 42149571, 42184803, 42204685])

#### 4. 매물 아이디로 매물 정보 가져오기

In [21]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain': 'zigbang',
    'item_ids': item_ids,
}
response = requests.post(url, json=params)
response

<Response [200]>

In [26]:
pd.options.display.max_columns = 40

In [39]:
data = response.json()['items']
df = pd.DataFrame(data)
df = df[df['address1'].str.contains(addr)].reset_index(drop=True)
df = df[['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 'floor_string', 'building_floor', 'title', 'manage_cost', 'address1']]
df.tail(1)

,item_id,sales_title,deposit,rent,size_m2,floor,floor_string,building_floor,title,manage_cost,address1
20,42232358,월세,300,73,17.0,3,3,3,"v망원한강코앞,화이트톤방,깔끔한 화장실,채광,조용한주택가",8,서울시 마포구 망원동


In [46]:
# 5. function
def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    geohash = geohash2.encode(lat, lng, precision=5)
    url = f'https://apis.zigbang.com/v2/items/oneroom?geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&domain=zigbang&checkAnyItemWithoutFilter=true'
    response = requests.get(url)
    item_ids = [data['itemId'] for data in response.json()['items']]
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain': 'zigbang',
        'item_ids': item_ids,
    }
    response = requests.post(url, json=params)
    data = response.json()['items']
    df = pd.DataFrame(data)
    df = df[df['address1'].str.contains(addr)].reset_index(drop=True)
    df = df[['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'manage_cost', 'address1']]
    return df

In [48]:
df = oneroom('개포동')
df

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1
0,40341496,전세,47000,0,53.09,5,5,"공간 활용 최상, 복층형 쓰리룸, 주차 가능",15,서울시 강남구 개포동
1,41090336,전세,35000,0,40.23,4,6,"가성비 최고의 선택, 깨끗하고 깔끔한 투룸, 즉시 입주 가능, 주차 가능",10,서울시 강남구 개포동
2,42183706,전세,35000,0,29.99,4,6,준신축 보증보험ok 방2화장실1 넓은사이즈 신혼집,6,서울시 강남구 개포동
3,42174824,전세,25450,0,28.56,5,6,버팀목 보증보험 대출 가능 신축급 고품격HOUSE,25,서울시 강남구 개포동
4,41949335,월세,5000,50,32.40,1,2,"깔끔한 투룸, 반려동물 가능한 매물",0,서울시 강남구 개포동
5,42168127,월세,4500,50,29.07,1,2,O 반려동물가능 l 공간활용 좋은 가성비투룸 O,0,서울시 강남구 개포동
6,42018349,전세,32500,0,29.98,4,5,전용큰2룸 거실뻥뷰 임대사업자 보증보험가능 풀옵션 붙박이장,7,서울시 강남구 개포동
7,42123845,전세,33000,0,46.57,4,5,"깨끗하고 편안한 공간 투룸, 즉시 입주 가능, 주차 가능, 필수 가전 완비",7,서울시 강남구 개포동
8,42123858,전세,33500,0,38.16,3,5,"탁 트인 채광 좋은 투룸, 즉시 입주 가능, 주차 가능, 풀옵션 완비",7,서울시 강남구 개포동
9,42160442,월세,500,60,16.53,2,5,인기매물 3중 보안 풀옵션 원룸,8,서울시 강남구 개포동


In [51]:
df = pd.DataFrame({'id' : list('456'), 'name' : list('ABC')})
df1 = df.iloc[::2]
df1.reset_index(drop=True)

,id,name
0,4,A
1,6,C


,item_id,section_type,deposit,rent,size_m2,floor,building_floor,title,manage_cost,address1
